In [281]:
import numpy as np 
import pandas as pd
from imblearn.combine import SMOTETomek
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import  GradientBoostingClassifier


import warnings
warnings.filterwarnings('ignore')

In [282]:
df=pd.read_csv("loan_prediction.csv")
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [283]:
df.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [284]:
numerical_features = df.select_dtypes(include = [np.number]).columns
categorical_features = df.select_dtypes(include = [np.object]).columns

In [285]:
df['Gender'] = df['Gender'].fillna(df['Gender'].mode()[0])
df['Married'] = df['Married'].fillna(df['Married'].mode()[0])
df['Dependents'] = df['Dependents'].str.replace('+','')
df['Dependents'] = df['Dependents'].fillna(df['Dependents'].mode()[0])
df['LoanAmount'] = df['LoanAmount'].fillna(df['LoanAmount'].mode()[0])
df['Self_Employed'] = df['Self_Employed'].fillna(df['Self_Employed'].mode()[0])
df['Loan_Amount_Term'] = df['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].mode()[0])
df['Credit_History'] = df['Credit_History'].fillna(df['Credit_History'].mode()[0])

In [286]:
df.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [287]:
df['Gender'].replace({'Male':1,'Female':0},inplace=True)
df['Dependents'].replace({'0':0,'1':1,'2':2,'3':3},inplace=True)
df['Married'].replace({'Yes':1,'No':0},inplace=True)
df['Self_Employed'].replace({'Yes':1,'No':0},inplace=True)
df['Property_Area'].replace({'Urban':2,'Rural':0,'Semiurban':1},inplace=True)
df['Education'].replace({'Graduate':1,'Not Graduate':0},inplace=True)
df['Loan_Status'].replace({'Y':1,'N':0},inplace=True)

In [288]:
df['CoapplicantIncome']=df['CoapplicantIncome'].astype("int64")
df['LoanAmount']=df['LoanAmount'].astype("int64")
df['Loan_Amount_Term']=df['Loan_Amount_Term'].astype("int64")
df['Credit_History']=df['Credit_History'].astype("int64")

In [289]:
le = LabelEncoder()
df['Loan_ID'] = le.fit_transform(df.Loan_ID)
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,0,1,0,0,1,0,5849,0,120,360,1,2,1
1,1,1,1,1,1,0,4583,1508,128,360,1,0,0
2,2,1,1,0,1,1,3000,0,66,360,1,2,1
3,3,1,1,0,0,0,2583,2358,120,360,1,2,1
4,4,1,0,0,1,0,6000,0,141,360,1,2,1


In [290]:
smote = SMOTETomek(0.90)

y = df['Loan_Status']
x = df.drop(columns=['Loan_Status'],axis=1)
x_bal,y_bal = smote.fit_resample(x,y)
print(y.value_counts())
print(y_bal.value_counts())

1    422
0    192
Name: Loan_Status, dtype: int64
1    359
0    316
Name: Loan_Status, dtype: int64


In [291]:
scaler = StandardScaler()
x_bal = scaler.fit_transform(x_bal)
x_bal = pd.DataFrame(x_bal)
x_bal.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,-1.728614,0.555103,-1.176928,-0.694247,0.635080,-0.337434,0.083439,-0.534258,-0.290496,0.267394,0.587045,1.346010
1,-1.722840,0.555103,0.849670,0.361196,0.635080,-0.337434,-0.131177,0.026120,-0.185757,0.267394,0.587045,-1.227805
2,-1.717065,0.555103,0.849670,-0.694247,0.635080,2.963547,-0.399532,-0.534258,-0.997487,0.267394,0.587045,1.346010
3,-1.711290,0.555103,0.849670,-0.694247,-1.574605,-0.337434,-0.470223,0.341983,-0.290496,0.267394,0.587045,1.346010
4,-1.705516,0.555103,-1.176928,-0.694247,0.635080,-0.337434,0.109037,-0.534258,-0.015556,0.267394,0.587045,1.346010


In [292]:
x_train, x_test, y_train, y_test = train_test_split(x_bal, y_bal, test_size = 0.33, random_state = 42)

# Compare the Models - KNN, Decision Tree, Random Forest CLassifier, XGBoost

## KNN

In [293]:
def kneighborsClassifier(x_train, x_test, y_train, y_test):
    knn = KNeighborsClassifier() 
    knn.fit(x_train,y_train)
    yPred = knn.predict(x_test)
    print("****KNN****") 
    print("Confusion matrix")
    print(confusion_matrix(y_test ,yPred) ) 
    print("Classification report")
    print(classification_report (y_test, yPred))
    y_pred=knn.predict(x_test)
    y_pred1=knn.predict(x_train)
    knn_test_accuracy = accuracy_score(y_test,y_pred)
    knn_train_accuracy = accuracy_score(y_train,y_pred1)
    print('Testing accuracy: ',knn_test_accuracy)
    print('Training accuracy: ', knn_train_accuracy)

kneighborsClassifier(x_train, x_test, y_train, y_test)

****KNN****
Confusion matrix
[[ 68  32]
 [ 21 102]]
Classification report
              precision    recall  f1-score   support

           0       0.76      0.68      0.72       100
           1       0.76      0.83      0.79       123

    accuracy                           0.76       223
   macro avg       0.76      0.75      0.76       223
weighted avg       0.76      0.76      0.76       223

Testing accuracy:  0.7623318385650224
Training accuracy:  0.827433628318584


# Decision Tree

In [294]:
def decisionTreeClassifier(x_train, x_test, y_train, y_test):
    dt = DecisionTreeClassifier() 
    dt.fit(x_train,y_train)
    yPred = dt.predict(x_test)
    print("****DecisionTreeClassifier****") 
    print("Confusion matrix")
    print(confusion_matrix(y_test ,yPred) ) 
    print("Classification report")
    print(classification_report (y_test, yPred))
    y_pred=dt.predict(x_test)
    y_pred1=dt.predict(x_train)
    decision_tree_test_accuracy = accuracy_score(y_test,y_pred)
    decision_tree_train_accuracy = accuracy_score(y_train,y_pred1)
    print('Testing accuracy: ', decision_tree_test_accuracy)
    print('Training accuracy: ', decision_tree_train_accuracy)

decisionTreeClassifier(x_train, x_test, y_train, y_test)

****DecisionTreeClassifier****
Confusion matrix
[[72 28]
 [26 97]]
Classification report
              precision    recall  f1-score   support

           0       0.73      0.72      0.73       100
           1       0.78      0.79      0.78       123

    accuracy                           0.76       223
   macro avg       0.76      0.75      0.75       223
weighted avg       0.76      0.76      0.76       223

Testing accuracy:  0.757847533632287
Training accuracy:  1.0


## Random Forest CLassifier

In [295]:
def randomForestClassifier(x_train, x_test, y_train, y_test):
    rf = RandomForestClassifier() 
    rf.fit(x_train,y_train)
    yPred = rf.predict(x_test)
    print("****RandomForestClassifier****") 
    print("Confusion matrix")
    print(confusion_matrix(y_test ,yPred) ) 
    print("Classification report")
    print(classification_report (y_test, yPred))
    y_pred=rf.predict(x_test)
    y_pred1=rf.predict(x_train)
    random_forest_test_accuracy = accuracy_score(y_test,y_pred)
    random_forest_train_accuracy = accuracy_score(y_train,y_pred1)
    print('Testing accuracy: ', random_forest_test_accuracy)
    print('Training accuracy: ',random_forest_train_accuracy)

randomForestClassifier(x_train, x_test, y_train, y_test)

****RandomForestClassifier****
Confusion matrix
[[ 69  31]
 [ 14 109]]
Classification report
              precision    recall  f1-score   support

           0       0.83      0.69      0.75       100
           1       0.78      0.89      0.83       123

    accuracy                           0.80       223
   macro avg       0.80      0.79      0.79       223
weighted avg       0.80      0.80      0.80       223

Testing accuracy:  0.7982062780269058
Training accuracy:  1.0


# XGBoost

In [296]:
def xgboost(x_train, x_test, y_train, y_test):
    xg = GradientBoostingClassifier() 
    xg.fit(x_train,y_train)
    yPred = xg.predict(x_test)
    print("****Gradient BoostingClassifier****") 
    print("Confusion matrix")
    print(confusion_matrix(y_test ,yPred) ) 
    print("Classification report")
    print(classification_report (y_test, yPred))
    y_pred=xg.predict(x_test)
    y_pred1=xg.predict(x_train)
    xgboost_test_accuracy = accuracy_score(y_test,y_pred)
    xgboost_train_accuracy = accuracy_score(y_train,y_pred1)
    print('Testing accuracy: ', xgboost_test_accuracy)
    print('Training accuracy: ', xgboost_train_accuracy)
  
xgboost(x_train, x_test, y_train, y_test)

****Gradient BoostingClassifier****
Confusion matrix
[[ 72  28]
 [ 14 109]]
Classification report
              precision    recall  f1-score   support

           0       0.84      0.72      0.77       100
           1       0.80      0.89      0.84       123

    accuracy                           0.81       223
   macro avg       0.82      0.80      0.81       223
weighted avg       0.81      0.81      0.81       223

Testing accuracy:  0.8116591928251121
Training accuracy:  0.9579646017699115


In [297]:
kneighborsClassifier(x_train, x_test, y_train, y_test)
decisionTreeClassifier(x_train, x_test, y_train, y_test)
randomForestClassifier(x_train, x_test, y_train, y_test)
xgboost(x_train, x_test, y_train, y_test)

****KNN****
Confusion matrix
[[ 68  32]
 [ 21 102]]
Classification report
              precision    recall  f1-score   support

           0       0.76      0.68      0.72       100
           1       0.76      0.83      0.79       123

    accuracy                           0.76       223
   macro avg       0.76      0.75      0.76       223
weighted avg       0.76      0.76      0.76       223

Testing accuracy:  0.7623318385650224
Training accuracy:  0.827433628318584
****DecisionTreeClassifier****
Confusion matrix
[[73 27]
 [26 97]]
Classification report
              precision    recall  f1-score   support

           0       0.74      0.73      0.73       100
           1       0.78      0.79      0.79       123

    accuracy                           0.76       223
   macro avg       0.76      0.76      0.76       223
weighted avg       0.76      0.76      0.76       223

Testing accuracy:  0.7623318385650224
Training accuracy:  1.0
****RandomForestClassifier****
Confusion matr

**KNN**
1.   Testing accuracy:  0.7955555555555556
2.   Training accuracy:  0.8392070484581498

**DecisionTreeClassifier**
1. Testing accuracy:  0.7822222222222223
2. Training accuracy:  1.0

**RandomForestClassifier**
1. Testing accuracy:  0.8133333333333334
2. Training accuracy:  1.0

**Gradient BoostingClassifier**
1. Testing accuracy:  0.8088888888888889
2. Training accuracy:  0.9405286343612335

RandomForestClassifier is chosen for further process.